In [138]:
import pandas as pd
import numpy as np
from datetime import datetime
import urllib.request, json # we will need urllib to communicate with the api and then json to read the data we get

### Get official NRW holidays (Feiertage) from API

In [139]:
with urllib.request.urlopen("https://feiertage-api.de/api/?jahr=2019") as url:
    nrw_holidays_json = json.loads(url.read().decode())

In [140]:
nrw_holidays = []
for name, data in nrw_holidays_json['NW'].items():
    nrw_holidays.append(pd.to_datetime(data['datum']))
index = pd.DatetimeIndex(nrw_holidays)
nrw_holidays = pd.Series(data=True, index = index)

### Get official NRW vacations (Ferien) from API

In [141]:
with urllib.request.urlopen("https://ferien-api.de/api/v1/holidays/NW/2019") as url:
    nrw_vacations_json = json.loads(url.read().decode())

In [142]:
nrw_vacations = pd.DataFrame(nrw_vacations_json)

In [143]:
nrw_vacations['start'] = pd.to_datetime(nrw_vacations['start'])
nrw_vacations['end'] = pd.to_datetime(nrw_vacations['end'])

In [144]:
nrw_vacations.drop(nrw_vacations.columns.difference(['start','end']), 1, inplace=True)

In [145]:
nrw_vacation_index = pd.DatetimeIndex({})
for index, row in nrw_vacations.iterrows():
    nrw_vacation_index = nrw_vacation_index.append(pd.date_range(row['start'], row['end']))

In [146]:
nrw_vacations = pd.Series(index=nrw_vacation_index, data=True)

In [147]:
nrw_holidays_full = pd.DataFrame({'is_holiday': nrw_holidays.append(nrw_vacations)})

We remove duplicate indeces, because holidays can be in vacations.

In [150]:
nrw_holidays_full = nrw_holidays_full.reset_index().drop_duplicates(subset='index', keep='last').set_index('index')

In [157]:
nrw_holidays_full = nrw_holidays_full.asfreq('h', fill_value=True)

In [158]:
nrw_holidays_full.to_pickle('../0_data/nrw_holidays.pkl')